**Autor:** __Tiago Dias__

In [2]:
!pip install wordninja
!pip install textblob

     |████████████████████████████████| 542kB 2.7MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-cp36-none-any.whl size=541553 sha256=c876a9f8914daa15a55b8e0ac4e448da925fa2fe7d810e873665077774c74e34
  Stored in directory: /root/.cache/pip/wheels/22/46/06/9b6d10ed02c85e93c3bb33ac50e2d368b2586248f192a2e22a
Successfully built wordninja


In [3]:
# Importação dos pacotes para Analise
import pandas as pd
import re
import numpy as np

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

import wordninja
import textblob
import collections
import tqdm

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from google.colab import files  

import warnings
warnings.simplefilter(action = 'ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Importação dos dataset
df = pd.read_csv('Train.csv')
df2 = pd.read_csv('Test.csv')

In [5]:
print('Tamanho do arquivo de Treino', df.shape)
print('Tamanho do arquivo de Teste', df2.shape)

Tamanho do arquivo de Treino (616, 3)
Tamanho do arquivo de Teste (309, 2)


In [7]:
df.head(2)

ID                                     text       label
0  SUAVK39Z  I feel that it was better I dieAm happy  Depression
1  9JDAGUV3             Why do I get hallucinations?       Drugs

In [8]:
df2.head(2)

ID                                       text
0  02V56KMO  How to overcome bad feelings and emotions
1  03BMGTOK             I feel like giving up in life

In [9]:
# Separando palavras juntas
df['text_split'] = df['text'].apply(wordninja.split)
df['text_new'] = df['text_split'].apply(TreebankWordDetokenizer().detokenize)
df.head(2)

In [11]:
# Corrigindo palavras incorretas
df['text_new'] = df['text_new'].apply(textblob.TextBlob).apply(textblob.TextBlob.correct).apply(str)
df.head(2)

In [12]:
df['label'].value_counts()

Depression    352
Alcohol       140
Suicide        66
Drugs          58
Name: label, dtype: int64

In [14]:
# Variáveis importantes
# Palavras para retirar da análise
stop_words = stopwords.words('english')
# Variável tamando da base de teste
test_size = 0.1
# Variáveis do modelo
epochs = 12
batch_size = 128

In [18]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['text_new'] = df['text_new'].str.replace('[0-9]+', '', regex=True).copy()
# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
df['text_new'] = df['text_new'].str.replace('[,.:;!?]+', ' ', regex=True).copy()
# Excluindo da descrição caracteres especiais, informações julgadas irrelevantes para a classificação.
df['text_new'] = df['text_new'].str.replace('[/<>()|\+\-\$%&#@\'\"]+', ' ', regex=True).copy()
# Colocando todos os caracteres em caixa baixa.
df['text_new'] = df['text_new'].str.lower().copy()
  
df.head(2)

ID  ...                      text_new
0  SUAVK39Z  ...  i feel better i die am happy
1  9JDAGUV3  ...      why i get hallucinations

[2 rows x 5 columns]

In [19]:
# Função para retirar stop words
def tokenize_df(tokenized_words):
  tokenized_words = word_tokenize(tokenized_words)
  stop = [word for word in tokenized_words if word not in stop_words]
  text = TreebankWordDetokenizer().detokenize(stop)
  return text

# Eliminando as stop words
df['text_new'] = df['text_new'].apply(tokenize_df).copy()
df.head(2)

ID  ...               text_new
0  SUAVK39Z  ...  feel better die happy
1  9JDAGUV3  ...     get hallucinations

[2 rows x 5 columns]

In [24]:
# Selecionando as unicas palavras da variável text_new
df['text_new_split'] = df['text_new'].apply(word_tokenize).copy()
text = list(df.text_new_split)
list_words = [item for sublist in text for item in sublist]
list_words = sorted(list_words)
only_words = set(list_words)
print('Quantidade de frases:', len(text), '\n')
print('Quantidade de GERAL palavras:', len(list_words), '\n')
print('Quantidade de UNICA palavras:', len(only_words), '\n')
counter = collections.Counter(list_words)

Quantidade de frases: 616 

Quantidade de GERAL palavras: 2293 

Quantidade de UNICA palavras: 711 



In [27]:
# Gerando encoder com o vocabulário das palavras
encoder = tfds.features.text.SubwordTextEncoder(vocab_list=only_words)
print('Vocabulary size(text_new): {}'.format(encoder.vocab_size))

Vocabulary size(text_new): 968


In [30]:
# Exemplo do encode e decode da text_new	
sample_string = 'feel better die happy'
# Encode
encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))
#Decode
original_string = encoder.decode(encoded_string)
print('The original string: "{}"'.format(original_string))

Encoded string is [349, 744, 345, 744, 392, 744, 475]
The original string: "feel better die happy"


In [32]:
# Validando a string original com a string após o decode
original_string == sample_string

True

In [33]:
# Exemplificando a relação de index ----> word
for index in encoded_string:
  print('{} ----> {}'.format(index, encoder.decode([index])))

349 ----> feel
744 ---->  
345 ----> better
744 ---->  
392 ----> die
744 ---->  
475 ----> happy


In [35]:
# Encode do label
label_encode = LabelEncoder()
target = label_encode.fit_transform(df['label'])

In [55]:
# Definindo feature e target
x = df['text_new']
y = target

# Dividindo dataset em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size)

In [56]:
# Criação da matriz
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

# Encode do input
def encode_input(text_new):
  list_x = []
  for text in text_new:
    text_encode = encoder.encode(text)
    text_encode = pad_to_size(text_encode, 64)
    list_x.append(text_encode)

  # Convertendo x em tensor
  input_encode = tf.cast(list_x, tf.float32)
  return input_encode

  # Encode do x_train e x_test
x_train = encode_input(x_train)
x_test = encode_input(x_test)

In [84]:
# Criação do modelo de RNN
int_sequences_input = tf.keras.Input(shape=(None,), dtype="int64")
embedded_sequences = tf.keras.layers.Embedding(encoder.vocab_size, 128)(int_sequences_input)
x = layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.25)(x)
preds = layers.Dense(4, activation="softmax")(x)
model = tf.keras.Model(int_sequences_input, preds)
model.summary()

Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_8 (Embedding)      (None, None, 128)         123904    
_________________________________________________________________
conv1d_24 (Conv1D)           (None, None, 128)         16512     
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_25 (Conv1D)           (None, None, 128)         16512     
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_26 (Conv1D)           (None, None, 128)       

In [85]:
# Compilando modelo e configurando o processo de treinamento
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=['accuracy'])

# Treinando o modelo
history = model.fit(x_train, y_train, epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(x_test, y_test))

Epoch 1/12
5/5 [==============================] - 0s 95ms/step - loss: 1.2790 - accuracy: 0.5217 - val_loss: 1.1965 - val_accuracy: 0.5000
Epoch 2/12
5/5 [==============================] - 0s 66ms/step - loss: 1.0441 - accuracy: 0.5794 - val_loss: 1.0339 - val_accuracy: 0.6290
Epoch 3/12
5/5 [==============================] - 0s 65ms/step - loss: 0.8192 - accuracy: 0.6841 - val_loss: 0.8144 - val_accuracy: 0.7258
Epoch 4/12
5/5 [==============================] - 0s 66ms/step - loss: 0.6285 - accuracy: 0.7581 - val_loss: 0.6981 - val_accuracy: 0.7258
Epoch 5/12
5/5 [==============================] - 0s 65ms/step - loss: 0.5021 - accuracy: 0.8105 - val_loss: 0.6511 - val_accuracy: 0.7258
Epoch 6/12
5/5 [==============================] - 0s 64ms/step - loss: 0.4180 - accuracy: 0.8303 - val_loss: 0.6388 - val_accuracy: 0.7097
Epoch 7/12
5/5 [==============================] - 0s 67ms/step - loss: 0.3367 - accuracy: 0.8863 - val_loss: 0.6095 - val_accuracy: 0.7742
Epoch 8/12
5/5 [===========

In [86]:
# Testando a qualidade do modelo
test_loss, test_acc = model.evaluate(x_test, y_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

2/2 [==============================] - 0s 5ms/step - loss: 0.7860 - accuracy: 0.7097
Test Loss: 0.7860071659088135
Test Accuracy: 0.7096773982048035


In [89]:
# Função para predição
def sample_predict(sample_pred_text):
  encoded_sample_pred_text = encoder.encode(sample_pred_text)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))
  return (predictions)

In [90]:
df2.head(2)

ID  ...                                   text_new
0  02V56KMO  ...  How to overcome bad feelings and emotions
1  03BMGTOK  ...              I feel like giving up in life

[2 rows x 4 columns]

In [91]:
sample_predict('How to overcome bad feelings and emotions')

array([[0.05074808, 0.3144645 , 0.0639507 , 0.57083666]], dtype=float32)

In [99]:
# Predição dos departamentos no df2
exemplo = 'How to overcome bad feelings and emotions'
predictions = sample_predict(exemplo)
probabilities = [np.argmax(predictions[0])]
# Retornando os departamentos
new_label = label_encode.inverse_transform(probabilities)
new_label[0]

'Suicide'